In [1]:
import pandas as pd

def read_updated_gdp_chile():
    file_path = '../data/pib sectorial y regional.xlsx'
    # Correctly load the dataset again, this time ensuring to skip only the first 2 rows
    df = pd.read_excel(file_path, skiprows=2)

    selected_columns = ['Periodo'] + [col for col in df.columns if col.startswith('PIB Reg')]
    df = df[selected_columns]
    def shorten_column_names(col):
        return col.split(',')[0]

# Apply the function to rename the columns
    df.columns = [shorten_column_names(col) for col in df.columns]
    df['Year'] = pd.to_datetime(df['Periodo']).dt.year
    df = df.drop('Periodo', axis=1)
    df = df[1:]
    df = agg_regions(df)
    return df

def agg_regions(df):
    df['I de Tarapacá'] = df['PIB Región de Arica y Parinacota'] + df['PIB Región de Tarapacá']
    df['VIII Del Biobío'] = df['PIB Región de Ñuble'] + df['PIB Región del Biobío']
    df['X De Los Lagos'] = df['PIB Región  de los Ríos'] + df['PIB Región de Los Lagos']
    df = df.drop(['PIB Región de Arica y Parinacota', 'PIB Región de Tarapacá', 'PIB Región de Ñuble',
                  'PIB Región del Biobío', 'PIB Región  de los Ríos', 'PIB Región de Los Lagos'], axis=1)
    return df

def read_updated_population_chile():
    file_path = '../data/poblacion regional.xlsx'
    # Correctly load the dataset again, this time ensuring to skip only the first 2 rows
    df = pd.read_excel(file_path, skiprows=2)

    # Apply the function to rename the columns
    df['Year'] = pd.to_datetime(df['Periodo']).dt.year
    df = df.drop('Periodo', axis=1)
    #df = agg_regions(df)
    reversed_new_region_names = {
        '2.Región de Tarapacá': 'I De Tarapacá',
        '3.Región de Antofagasta': 'II De Antofagasta',
        '4.Región de Atacama': 'III De Atacama',
        '5.Región de Coquimbo': 'IV De Coquimbo',
        '6.Región de Valparaíso': 'V De Valparaíso',
        '7.Región Metropolitana': 'RMS Región Metropolitana de Santiago',
        '8.Región del Libertador General Bernardo O\'Higgins': 'VI Del Libertador General Bernardo OHiggins',
        '9.Región del Maule': 'VII Del Maule',
        '10.Región del Biobío': 'VIII Del Biobío',
        '12.Región de La Araucanía': 'IX De La Araucanía',
        '14.Región de Los Lagos': 'X De Los Lagos',
        '15.Región de Aysén del General Carlos Ibáñez del Campo': 'XI Aysén del General Carlos Ibáñez del Campo',
        '16.Región de Magallanes y la Antártica Chilena': 'XII De Magallanes y de la Antártica Chilena'
    }

    df = df.rename(columns=reversed_new_region_names)
    df['I De Tarapacá'] = df['1.Región de Arica y Parinacota'] + df['I De Tarapacá']
    df['VIII Del Biobío'] = df['11.Región de Ñuble'] + df['VIII Del Biobío']
    df['X De Los Lagos'] = df['13.Región de Los Ríos'] + df['X De Los Lagos']
    df = df.drop(['1.Región de Arica y Parinacota', '11.Región de Ñuble', '13.Región de Los Ríos'], axis=1)
    return df

def read_and_clean_data():
    # Read the datasets
    gdp = read_updated_gdp_chile()
    pop = read_updated_population_chile()

# Clean and align the column names to keep only the region numbers and the 'Year' column
def clean_gdp_column_names(df):
    df = df.rename(columns={
        'PIB Región de Antofagasta': 'II De Antofagasta',
        'PIB Región de Atacama': 'III De Atacama',
        'PIB Región de Coquimbo': 'IV De Coquimbo',
        'PIB Región de Valparaíso': 'V De Valparaíso',
        'PIB Región Metropolitana de Santiago': 'RMS Región Metropolitana de Santiago',
        'PIB Región del Libertador General Bernardo OHiggins': 'VI Del Libertador General Bernardo OHiggins',
        'PIB Región del Maule': 'VII Del Maule',
        'PIB Región de La Araucanía': 'IX De La Araucanía',
        'PIB Región de Aysén del General Carlos Ibáñez del Campo': 'XI Aysén del General Carlos Ibáñez del Campo',
        'PIB Región de Magallanes y de la Antártica Chilena': 'XII De Magallanes y de la Antártica Chilena',
        'I de Tarapacá': 'I De Tarapacá',
        'VIII Del Biobío': 'VIII Del Biobío',
        'X De Los Lagos': 'X De Los Lagos'
    })
    return df

def calculate_gdp_per_capita():
    gdp = read_updated_gdp_chile()
    pop = read_updated_population_chile()
    gdp_clean = clean_gdp_column_names(gdp)

    # Merge datasets on 'Year'
    merged_df = pd.merge(gdp_clean, pop, on='Year', suffixes=('_gdp', '_pop'))

    # Calculate GDP per capita
    gdp_per_capita = pd.DataFrame()
    gdp_per_capita['Year'] = merged_df['Year']

    # Calculate GDP per capita for each region
    regions = [
        'I De Tarapacá', 'II De Antofagasta', 'III De Atacama',
        'IV De Coquimbo', 'V De Valparaíso',
        'RMS Región Metropolitana de Santiago',
        'VI Del Libertador General Bernardo OHiggins', 'VII Del Maule',
        'VIII Del Biobío', 'IX De La Araucanía', 'X De Los Lagos',
        'XI Aysén del General Carlos Ibáñez del Campo',
        'XII De Magallanes y de la Antártica Chilena'
    ]

    for region in regions:
        gdp_per_capita['GDP per capita ' + region] = merged_df[region + "_gdp"] * 10E8 / merged_df[region + "_pop"]
    return gdp_per_capita



In [2]:
gdp_per_capita = calculate_gdp_per_capita()
gdp_per_capita.head()

,Year,GDP per capita I De Tarapacá,GDP per capita II De Antofagasta,GDP per capita III De Atacama,GDP per capita IV De Coquimbo,GDP per capita V De Valparaíso,GDP per capita RMS Región Metropolitana de Santiago,GDP per capita VI Del Libertador General Bernardo OHiggins,GDP per capita VII Del Maule,GDP per capita VIII Del Biobío,GDP per capita IX De La Araucanía,GDP per capita X De Los Lagos,GDP per capita XI Aysén del General Carlos Ibáñez del Campo,GDP per capita XII De Magallanes y de la Antártica Chilena
0,2013.0,9565680.200294,26648642.096778,12078110.852386,7560764.558379,7387770.326832,9915392.093437,7497698.775946,5703655.168489,5830797.913813,4470174.916803,5665132.075419,10694246.308066,9593701.385251
1,2014.0,9620103.462684,27229871.137882,12423529.522398,7153965.152334,7116097.656261,9999328.624198,7573141.756477,5672800.807762,5969577.093571,4511948.725622,5987121.274582,10998648.515007,10086012.402995
2,2015.0,9551996.245038,26769329.082479,12249570.231358,6977004.476096,7065382.912088,10135110.895673,7897623.939096,6058175.863929,6115560.252341,4693856.090734,6072571.054874,10346902.201293,10232196.989101
3,2016.0,9194277.668792,25074058.641426,12579769.864771,6979682.834087,7206690.28267,10242235.692777,7980947.619308,6105844.652599,6217989.714982,4914866.160753,6215580.304468,10309666.796909,10671132.815884
4,2017.0,9413714.939906,23950186.336171,12440400.952811,6981626.701849,7237440.857219,10162455.418273,7790062.119079,6056066.618138,6284267.478803,4982030.971457,6501457.879382,11296396.237783,10803056.344823
